In [1]:
# Add "parameters" (jupyter-notebook) tag to this cell, to allow papermill to inject different parameters
from datetime import date
it=0  #Iteration of gridsearch
# to put it all in one folder by date, will be replaced by papermill
today=date.today() 
rdate=today.strftime("%Y-%m-%d")
rdate="2021-08-31"
# Training Params
best_runs = []
training_stats = {}
epochs = 15
loops = 5

In [2]:
# Parameters
it = 12
rdate = "2021-08-31"


In [3]:
# See if running on Colab (for setting the correct workdir and installing all dependencies)
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  g_colab = True
else:
  print('Not running on CoLab')
  g_colab = False

Not running on CoLab


In [4]:
# connect to drive
if g_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    %cd "/gdrive/MyDrive/1 Job/Product and Code/CogAlex 2.0/"

# Choose model
# Gridsearch parameters
from sklearn.model_selection import ParameterGrid

# Original results with "xlm_roberta_base"
grid = [{"model_name": ["xlm-roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new"], ["de_train_new"], ["de_val_new"],
                      ["en_train_new", "en_val_new"], ["en_train_new"], ["en_val_new"], 
                      ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]},
        {"model_name": ["xlm-roberta-large", "distilbert-base-multilingual-cased", "bert-base-multilingual-uncased", "bert-base-multilingual-cased", "roberta-base"],
        "datasets": [["old"], ["de_train_new", "de_val_new","en_train_new", "en_val_new"]]}]

pg = list(ParameterGrid(grid))
print(len(pg))
model_name = pg[it]["model_name"]
datasets = pg[it]["datasets"]
print(pg[it])

18
{'datasets': ['old'], 'model_name': 'roberta-base'}


# Manually set parameters

In [5]:
# Savedir
import os
savedir = f"./{rdate}/averages/{model_name}_{it}_{datasets}"
if g_colab:
    workdir = os.getcwd()
else:
    workdir = os.getcwd().replace("/home/","/binfl/")
model_dir = f"{workdir}/saved_models/{rdate}"

# Check for GPU support
import torch
if torch.cuda.is_available():
        print("Using GPU for inference")
        print(torch.cuda.get_device_name(torch.cuda.current_device()))
        device = torch.cuda.current_device()
else:
        print("Using CPU for inference")
        device = -1

Using GPU for inference
GeForce GTX 1080


In [6]:
if g_colab:
    !pip install transformers
    !pip install sentencepiece

# Libraries

In [7]:
import torch                                              #for training the model
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
import pandas as pd                                       #for handling the data
from transformers import XLMRobertaTokenizer, AutoTokenizer              #for loading the pretrained model and tokenizer
from transformers import XLMRobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW                            
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from sklearn import preprocessing                         #for label encoding
from sklearn.metrics import classification_report         #for showing performance on validation/test sets
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid         #for grid search
from sklearn.model_selection import ParameterSampler      #for random search
from sklearn.utils.fixes import loguniform
import sentencepiece
import matplotlib.pyplot as plt
import time
import datetime
import random
import numpy as np
%matplotlib inline
import seaborn as sns
# For file saving etc.
import os
import shutil

# Load Data

In [8]:
# sk learn label encoder for changing the labels to integers
labels=["ANT", "HYP", "RANDOM", "SYN"]
le = preprocessing.LabelEncoder()
le.fit(labels)
print(labels)

['ANT', 'HYP', 'RANDOM', 'SYN']


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
if "gpt2" in model_name:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token_id

1

# Evaluate Validation

In [10]:
label_dict = {i : l for i,l in enumerate(labels)}
print(label_dict)

{0: 'ANT', 1: 'HYP', 2: 'RANDOM', 3: 'SYN'}


In [11]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "all":0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "all":""}
for run in os.listdir(model_dir):
    print(run + "\n")
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",
                                                               num_labels=4,  
                                                               id2label=label_dict)                                                        
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    validgold_zh, validgold_de, validgold_en = [open(f"./datasets/validgold_{l}_data.txt").readlines() for l in ["chinese", "german", "english"]]
    validgold_all =   validgold_zh + validgold_de + validgold_en 
    for data, read_lang in zip([validgold_zh, validgold_de, validgold_en, validgold_all], ["chinese", "german", "english", "all"]):
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict

Run 3



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.68      0.69      0.68       136
         HYP       0.79      0.57      0.66       145
      RANDOM       0.78      0.91      0.84       428
         SYN       0.88      0.64      0.74       129

    accuracy                           0.77       838
   macro avg       0.78      0.70      0.73       838
weighted avg       0.78      0.77      0.77       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.82      0.69      0.75       136
         HYP       0.86      0.57      0.68       145
         SYN       0.92      0.64      0.76       129

   micro avg       0.87      0.63      0.73       410
   macro avg       0.87      0.63      0.73       410
weighted avg       0.87      0.63      0.73       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.29      0.16      0.21       275
         HYP       0.31      0.15      0.21       294
      RANDOM       0.54      0.78      0.64       786
         SYN       0.26      0.18      0.22       272

    accuracy                           0.46      1627
   macro avg       0.35      0.32      0.32      1627
weighted avg       0.41      0.46      0.42      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.45      0.16      0.24       275
         HYP       0.54      0.15      0.24       294
         SYN       0.37      0.18      0.25       272

   micro avg       0.44      0.17      0.24       841
   macro avg       0.45      0.17      0.24       841
weighted avg       0.46      0.17      0.24       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.53      0.51      0.52       308
         HYP       0.45      0.43      0.44       292
      RANDOM       0.78      0.73      0.75       877
         SYN       0.38      0.49      0.43       259

    accuracy                           0.61      1736
   macro avg       0.53      0.54      0.54      1736
weighted avg       0.62      0.61      0.61      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.75      0.51      0.61       308
         HYP       0.59      0.43      0.50       292
         SYN       0.50      0.49      0.49       259

   micro avg       0.61      0.48      0.53       859
   macro avg       0.61      0.48      0.53       859
weighted avg       0.62      0.48      0.54       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.50      0.41      0.45       719
         HYP       0.48      0.35      0.40       731
      RANDOM       0.67      0.79      0.72      2091
         SYN       0.42      0.39      0.41       660

    accuracy                           0.58      4201
   macro avg       0.52      0.48      0.50      4201
weighted avg       0.57      0.58      0.57      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.70      0.41      0.52       719
         HYP       0.65      0.35      0.45       731
         SYN       0.54      0.39      0.46       660

   micro avg       0.62      0.38      0.48      2110
   macro avg       0.63      0.38      0.48      2110
weighted avg       0.63      0.38      0.48      2110

Run 5



...

Results for all: 

              precision    recall  f1-score   support

         ANT       0.50      0.41      0.45       719
         HYP       0.48      0.35      0.40       731
      RANDOM       0.67      0.79      0.72      2091
         SYN       0.42      0.39      0.41       660

    accuracy                           0.58      4201
   macro avg       0.52      0.48      0.50      4201
weighted avg       0.57      0.58      0.57      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.70      0.41      0.52       719
         HYP       0.65      0.35      0.45       731
         SYN       0.54      0.39      0.46       660

   micro avg       0.62      0.38      0.48      2110
   macro avg       0.63      0.38      0.48      2110
weighted avg       0.63      0.38      0.48      2110

Run 1



Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.62      0.78      0.69       136
         HYP       0.81      0.72      0.76       145
      RANDOM       0.86      0.86      0.86       428
         SYN       0.84      0.75      0.80       129

    accuracy                           0.80       838
   macro avg       0.79      0.78      0.78       838
weighted avg       0.81      0.80      0.80       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.80      0.78      0.79       136
         HYP       0.91      0.72      0.80       145
         SYN       0.92      0.75      0.83       129

   micro avg       0.87      0.75      0.81       410
   macro avg       0.88      0.75      0.81       410
weighted avg       0.88      0.75      0.81       410



Results for german: 

              precision    recall  f1-score   support

         ANT       0.26      0.23      0.25       275
         HYP       0.25      0.22      0.23       294
      RANDOM       0.55      0.63      0.59       786
         SYN       0.26      0.22      0.24       272

    accuracy                           0.42      1627
   macro avg       0.33      0.33      0.33      1627
weighted avg       0.40      0.42      0.41      1627
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.45      0.23      0.31       275
         HYP       0.44      0.22      0.29       294
         SYN       0.39      0.22      0.29       272

   micro avg       0.43      0.22      0.29       841
   macro avg       0.43      0.22      0.29       841
weighted avg       0.43      0.22      0.29       841



Results for english: 

              precision    recall  f1-score   support

         ANT       0.53      0.53      0.53       308
         HYP       0.42      0.45      0.43       292
      RANDOM       0.78      0.74      0.76       877
         SYN       0.41      0.43      0.42       259

    accuracy                           0.61      1736
   macro avg       0.53      0.54      0.53      1736
weighted avg       0.62      0.61      0.61      1736
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.77      0.53      0.63       308
         HYP       0.56      0.45      0.50       292
         SYN       0.49      0.43      0.46       259

   micro avg       0.60      0.47      0.53       859
   macro avg       0.61      0.47      0.53       859
weighted avg       0.62      0.47      0.53       859



Results for all: 

              precision    recall  f1-score   support

         ANT       0.46      0.46      0.46       719
         HYP       0.43      0.41      0.42       731
      RANDOM       0.70      0.72      0.71      2091
         SYN       0.43      0.41      0.42       660

    accuracy                           0.57      4201
   macro avg       0.51      0.50      0.50      4201
weighted avg       0.57      0.57      0.57      4201
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.68      0.46      0.55       719
         HYP       0.61      0.41      0.49       731
         SYN       0.55      0.41      0.47       660

   micro avg       0.62      0.43      0.50      2110
   macro avg       0.62      0.43      0.50      2110
weighted avg       0.62      0.43      0.50      2110



In [12]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results.loc[k] = temp_df.mean()
avg_val_results.update(avg_val_results.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results

,precision,recall,f1-score,support
chinese,78.6,78.0,77.6,838.0
german,40.8,45.5,41.6,1627.0
english,61.9,60.6,61.1,1736.0
all,56.8,58.2,57.1,4201.0


In [13]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_val_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_val_results_no_random.loc[k] = temp_df.mean()
avg_val_results_no_random.update(avg_val_results_no_random.loc[:,[i for i in avg_val_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_val_results_no_random

,precision,recall,f1-score,support
chinese,87.1,65.5,74.5,410.0
german,44.9,17.8,25.2,841.0
english,62.0,47.7,53.6,859.0
all,62.9,39.3,48.2,2110.0


In [14]:
# save average scores
os.makedirs(savedir, exist_ok=True)
val_preds_savedir = savedir + "/val_preds"
os.makedirs(val_preds_savedir, exist_ok=True)
avg_val_results.to_csv(f"{savedir}/val_avg_{model_name}_{it}.csv")
avg_val_results_no_random.to_csv(f"{savedir}/val_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(val_preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        

# Evaluate on Gold

In [15]:
results = {}
results_no_random = {}
weighted_F1_scores = {"chinese":0, "german":0, "english":0, "italian":0, "german NEW":0, "english NEW": 0}
preds_per_lang = {"chinese":"", "german":"", "english":"", "italian":"", "german NEW":"", "english NEW": ""}
for run in os.listdir(model_dir):
    results[run] = {}
    results_no_random[run] = {}
    model = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{run}/{model_name}_{it}",  id2label=label_dict)
    model.eval()
    annotate = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    gold_zh, gold_de, gold_en, gold_it = [open(f"./datasets/gold_{l}_data.txt").readlines() for l in ["chinese", "german", "english", "italian"]]
    gold_de_new, gold_en_new = [open(f"./datasets/gold_{l}_data_new.txt").readlines() for l in ["german", "english"]]   
    for data, read_lang in zip([gold_zh, gold_de, gold_en, gold_it, gold_de_new, gold_en_new], ["chinese", "german", "english", "italian", "german NEW", "english NEW"]):
        # Load data for prediction/scoring
        test_set = [line.split("\t") for line in data]
        pred_input = [line[0].strip() + tokenizer.sep_token * 2 + line[1].strip() for line in test_set]
        gold_labels = [line[2].strip() for line in test_set]
        
        #Predict labels on test_set
        pred_list = annotate(pred_input)
        pred_labels = [entry["label"] for entry in pred_list]
        report = classification_report(gold_labels, pred_labels)
        report_dict = classification_report(gold_labels, pred_labels, output_dict=True)
        print(f"Results for {read_lang}: \n")
        print(report, "\n\n")
        results[run][read_lang]= report_dict
        
        # Save predictions
        weighted_F1 = f1_score(gold_labels, pred_labels, average='weighted')
        if weighted_F1 > weighted_F1_scores[read_lang]:
            pred_with_words = []
            for line_words, line_pred in zip(test_set, pred_labels):
                pred_with_words.append("\t".join(line_words[:2]) + "\t" + str(line_pred))
            preds_per_lang[read_lang] = pred_with_words
            weighted_F1_scores[read_lang] = f1_score(gold_labels, pred_labels, average='weighted')
        
        # No RANDOM scores, the CogALex way:
        for i in range(len(gold_labels) -1, -1, -1):
            if gold_labels[i] == 'RANDOM':
                gold_labels.pop(i)
                pred_labels.pop(i)
        lbls = ("ANT", "HYP", "SYN")  
        report_no_random = classification_report(gold_labels, pred_labels, labels=lbls)
        report_no_random_dict = classification_report(gold_labels, pred_labels, labels=lbls, output_dict=True)
        print("\nNo RANDOM:\n")
        print(report_no_random)
        results_no_random[run][read_lang] = report_no_random_dict
        #print(results_no_random[run][read_lang]["weighted avg"])

Results for chinese: 

              precision    recall  f1-score   support

         ANT       0.67      0.64      0.66       142
         HYP       0.85      0.64      0.73       129
      RANDOM       0.79      0.90      0.84       445
         SYN       0.83      0.70      0.76       122

    accuracy                           0.79       838
   macro avg       0.79      0.72      0.75       838
weighted avg       0.79      0.79      0.78       838
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.84      0.64      0.73       142
         HYP       0.93      0.64      0.76       129
         SYN       0.91      0.70      0.79       122

   micro avg       0.89      0.66      0.76       393
   macro avg       0.90      0.66      0.76       393
weighted avg       0.89      0.66      0.76       393



Results for german: 

              precision    recall  f1-score   support

         ANT       0.41      0.21      0.27       281
         HYP       0.31      0.20      0.24       286
      RANDOM       0.57      0.80      0.66       796
         SYN       0.31      0.20      0.24       265

    accuracy                           0.50      1628
   macro avg       0.40      0.35      0.36      1628
weighted avg       0.45      0.50      0.45      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.55      0.21      0.30       281
         HYP       0.51      0.20      0.29       286
         SYN       0.43      0.20      0.27       265

   micro avg       0.49      0.20      0.29       832
   macro avg       0.50      0.20      0.29       832
weighted avg       0.50      0.20      0.29       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.53      0.52      0.52       306
         HYP       0.44      0.41      0.43       279
      RANDOM       0.80      0.77      0.78       887
         SYN       0.38      0.48      0.43       266

    accuracy                           0.62      1738
   macro avg       0.54      0.54      0.54      1738
weighted avg       0.63      0.62      0.63      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.73      0.52      0.61       306
         HYP       0.57      0.41      0.48       279
         SYN       0.48      0.48      0.48       266

   micro avg       0.59      0.47      0.52       851
   macro avg       0.59      0.47      0.52       851
weighted avg       0.60      0.47      0.52       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.32      0.26      0.29       144
         HYP       0.16      0.05      0.07       153
      RANDOM       0.59      0.44      0.50       523
         SYN       0.22      0.53      0.31       187

    accuracy                           0.37      1007
   macro avg       0.32      0.32      0.29      1007
weighted avg       0.42      0.37      0.37      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.49      0.26      0.34       144
         HYP       0.33      0.05      0.08       153
         SYN       0.44      0.53      0.48       187

   micro avg       0.44      0.30      0.36       484
   macro avg       0.42      0.28      0.30       484
weighted avg       0.42      0.30      0.31       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.38      0.20      0.26       281
         HYP       0.35      0.22      0.27       286
      RANDOM       0.57      0.81      0.67       796
         SYN       0.32      0.18      0.23       265

    accuracy                           0.50      1628
   macro avg       0.40      0.36      0.36      1628
weighted avg       0.45      0.50      0.46      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.50      0.20      0.29       281
         HYP       0.58      0.22      0.32       286
         SYN       0.43      0.18      0.26       265

   micro avg       0.50      0.20      0.29       832
   macro avg       0.50      0.20      0.29       832
weighted avg       0.51      0.20      0.29       832



...

Results for german: 

              precision    recall  f1-score   support

         ANT       0.34      0.30      0.32       281
         HYP       0.27      0.29      0.28       286
      RANDOM       0.55      0.60      0.57       796
         SYN       0.26      0.20      0.22       265

    accuracy                           0.43      1628
   macro avg       0.35      0.35      0.35      1628
weighted avg       0.42      0.43      0.42      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.54      0.30      0.39       281
         HYP       0.49      0.29      0.37       286
         SYN       0.44      0.20      0.27       265

   micro avg       0.50      0.27      0.35       832
   macro avg       0.49      0.26      0.34       832
weighted avg       0.49      0.27      0.34       832



Results for english: 

              precision    recall  f1-score   support

         ANT       0.55      0.54      0.54       306
         HYP       0.41      0.42      0.41       279
      RANDOM       0.80      0.79      0.80       887
         SYN       0.43      0.46      0.45       266

    accuracy                           0.64      1738
   macro avg       0.55      0.55      0.55      1738
weighted avg       0.64      0.64      0.64      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.74      0.54      0.62       306
         HYP       0.52      0.42      0.47       279
         SYN       0.52      0.46      0.49       266

   micro avg       0.59      0.47      0.53       851
   macro avg       0.59      0.47      0.53       851
weighted avg       0.60      0.47      0.53       851



Results for italian: 

              precision    recall  f1-score   support

         ANT       0.26      0.40      0.31       144
         HYP       0.21      0.10      0.13       153
      RANDOM       0.61      0.42      0.50       523
         SYN       0.24      0.44      0.31       187

    accuracy                           0.37      1007
   macro avg       0.33      0.34      0.31      1007
weighted avg       0.43      0.37      0.38      1007
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.45      0.40      0.42       144
         HYP       0.43      0.10      0.16       153
         SYN       0.46      0.44      0.45       187

   micro avg       0.45      0.32      0.37       484
   macro avg       0.44      0.31      0.34       484
weighted avg       0.45      0.32      0.35       484



Results for german NEW: 

              precision    recall  f1-score   support

         ANT       0.35      0.33      0.34       281
         HYP       0.30      0.32      0.31       286
      RANDOM       0.57      0.62      0.59       796
         SYN       0.25      0.19      0.22       265

    accuracy                           0.45      1628
   macro avg       0.37      0.36      0.37      1628
weighted avg       0.43      0.45      0.44      1628
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.53      0.33      0.41       281
         HYP       0.54      0.32      0.40       286
         SYN       0.42      0.19      0.26       265

   micro avg       0.51      0.28      0.36       832
   macro avg       0.50      0.28      0.36       832
weighted avg       0.50      0.28      0.36       832



Results for english NEW: 

              precision    recall  f1-score   support

         ANT       0.57      0.57      0.57       306
         HYP       0.46      0.48      0.47       283
      RANDOM       0.83      0.80      0.82       887
         SYN       0.45      0.49      0.47       262

    accuracy                           0.66      1738
   macro avg       0.58      0.59      0.58      1738
weighted avg       0.67      0.66      0.67      1738
 



No RANDOM:

              precision    recall  f1-score   support

         ANT       0.76      0.57      0.65       306
         HYP       0.58      0.48      0.53       283
         SYN       0.53      0.49      0.51       262

   micro avg       0.62      0.52      0.56       851
   macro avg       0.62      0.51      0.56       851
weighted avg       0.63      0.52      0.56       851



In [16]:
results_by_lang = {}

for k1 in results.keys():
    for k2 in results[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results.loc[k] = temp_df.mean()
avg_test_results.update(avg_test_results.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results

,precision,recall,f1-score,support
chinese,79.0,78.7,78.5,838.0
german,44.5,48.3,44.8,1628.0
english,63.4,62.4,62.8,1738.0
italian,42.0,37.1,37.3,1007.0
german NEW,45.0,48.9,45.3,1628.0
english NEW,66.3,65.1,65.6,1738.0


In [17]:
results_by_lang = {}

for k1 in results_no_random.keys():
    for k2 in results_no_random[k1].keys():
        if k2 in results_by_lang.keys():
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
        else:
            results_by_lang[k2] = {}
            results_by_lang[k2][k1] = results_no_random[k1][k2]["weighted avg"]
columns = [i for i in results_by_lang.keys()]
avg_test_results_no_random = pd.DataFrame(columns=["precision", "recall", "f1-score", "support"])
for k in columns:
    temp_df = pd.DataFrame(results_by_lang[k]).transpose()
    avg_test_results_no_random.loc[k] = temp_df.mean()
avg_test_results_no_random.update(avg_test_results_no_random.loc[:,[i for i in avg_test_results.keys() if i != "support"]].apply(lambda x: round(x * 100, 1)))
avg_test_results_no_random

,precision,recall,f1-score,support
chinese,89.5,68.0,77.2,393.0
german,49.7,21.5,29.8,832.0
english,60.0,47.2,52.6,851.0
italian,42.5,30.4,32.1,484.0
german NEW,50.4,22.0,30.5,832.0
english NEW,63.2,51.4,56.4,851.0


In [18]:
# save average scores
os.makedirs(savedir, exist_ok=True)
preds_savedir = savedir + "/preds"
os.makedirs(preds_savedir, exist_ok=True)
avg_test_results.to_csv(f"{savedir}/test_avg_{model_name}_{it}.csv")
avg_test_results_no_random.to_csv(f"{savedir}/test_no_random_avg_{model_name}_{it}.csv")
for key in preds_per_lang.keys():
    with open(preds_savedir + f"/{key}-predictions.txt", "w", encoding="utf-8") as f:
        for line in preds_per_lang[key]:
            print(line, file=f)        